# Generate top of the book & trades using dedicated service - Python

### Overview
Retrieving trades and top of the book data points over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request simultaneously trades and top of the book and illustrate market activity over a look back period.

This sample demonstrates how to request and plot from a dedicated tick data service **on-demand** trades and best bid/ask over the time.

### Inputs/outputs
Trades and best Bid/Ask tick data extraction sample requires instrument's identifier, date time intervals as per inputs and returns top fo the book the occured trades data points.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and top of the book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and top of the book tick data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* and *plotly* as per display packages
    
***

# Run Top of the book & Trades sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly

In [ ]:
pip install -i https://test.pypi.org/simple/ systemathics.apis==0.9.27

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2 as tick_trades_and_book
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2_grpc as tick_trades_and_book_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve data
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick trades and book request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 3, day = 5), 
    end_date = date.Date(year = 2021, month = 3, day = 5)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 14, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 14, minutes = 30, seconds = 0)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate the tick trades and book request
identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_trades_and_book.TickTradesAndBookRequest(
    identifiers = [identifier],
    constraints = constraints
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the tick trades and book service
    service = tick_trades_and_book_service.TickTradesAndBookServiceStub(channel)
    
    # process the tick trades and book request
    time_stamps, trade_prices, trade_sizes, bid_prices, bid_sizes, ask_prices, ask_sizes  = [], [], [],[], [], [], []
    limits_count, trades_count = 0, 0

    for current in service.TickTradesAndBook(request = request, metadata = [('authorization', token)]):
        # get timestamps
        if current.data.time_stamp.seconds > 0:
            time_stamps.append(current.data.time_stamp)
        # get book limits and set trade fields to nan
        if current.data.book.time_stamp.seconds > 0:
            bid_prices.append(current.data.book.bid.price)
            bid_sizes.append(current.data.book.bid.size)
            ask_prices.append(current.data.book.ask.price)
            ask_sizes.append(- current.data.book.ask.size)
            trade_prices.append(None)
            trade_sizes.append(None)
            limits_count += 1
        # get trades and set book limits fields to nan
        if current.data.trade.time_stamp.seconds > 0:
            trade_prices.append(current.data.trade.price)
            trade_sizes.append(current.data.trade.size)
            bid_prices.append(None)
            bid_sizes.append(None)
            ask_prices.append(None)
            ask_sizes.append(None)
            trades_count += 1

The following code snippet displays the total retrieved trades and book limits counts:

In [ ]:
print('Total trades retrieved: {0}'.format(trades_count))
print('Total book limits retrieved: {0}'.format(limits_count))

### Step 4: Visualize data

#### 4.1 Retrieve data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# prepare the dataframe content for trades and book limits
dates = [datetime.fromtimestamp(ts.seconds) for ts in time_stamps]
# create a pandas dataframe with: dates, trades and limits
d = {'Date': dates, 'Trade price': trade_prices, 'Trade size': trade_sizes, 'Bid size': bid_sizes, 'Bid price': bid_prices, 'Ask price': ask_prices, 'Ask size': ask_sizes}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot trades and top of the book with plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# plot trades and top of the book prices
fig = make_subplots()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Bid price'], name='Bid price', marker_color ='green'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Ask price'], name='Ask price', marker_color ='red'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Trade price'], name='Trade price', marker_color ='blue'))
fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Tick trades and best bid/ask prices for {0}-{1}'.format(ticker,exchange))
fig.show()

# plot trades and top of the book sizes
fig = make_subplots()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Bid size'], name='Bid size', marker_color ='green'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Ask size'], name='Ask size', marker_color ='red'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Trade size'], name='Ask size', marker_color ='blue', mode='markers'))
fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Tick trades and best bid/ask sizes for {0}-{1}'.format(ticker,exchange))
fig.show()
